In [11]:
import asyncio
import gspread
import pandas as pd
from playwright.async_api import async_playwright
from google.oauth2.service_account import Credentials
from gspread.utils import rowcol_to_a1

# =========================
# GOOGLE AUTH
# =========================
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file(
    "/Users/zentrovoy/Documents/Insight Genie/HR-GAME/My Genie game/CascadeProjects/piplines/genie-game.json",
    scopes=scopes
)
client = gspread.authorize(creds)

# MAIN SHEET DOCUMENT
doc = client.open_by_key("1lG0_Y_o2jl2rackWJyB1esAUcINb843PhNZBmTwf2L4")

# Sheets
sheet_game = doc.worksheet("Sheet1")          # ONLY telemetry
sheet_facets = doc.worksheet("OCEAN_Facets")  # ONLY Big Five + Facets (Real/Pred/Error)

# Load both sheets into DataFrames
header_game = sheet_game.row_values(1)
df_game = pd.DataFrame(sheet_game.get_all_records(expected_headers=header_game))

header_facets = sheet_facets.row_values(1)
df_facets = pd.DataFrame(sheet_facets.get_all_records(expected_headers=header_facets))

# =========================
# TRAIT & FACET ORDER EXACTLY LIKE SITE
# =========================
TRAITS = [
    "Trait_Neuroticism",
    "Trait_Extraversion",
    "Trait_Openness",
    "Trait_Agreeableness",
    "Trait_Conscientiousness",
]

FACETS = [
    "Facet_Anxiety", "Facet_Anger", "Facet_Depression", "Facet_SelfConsciousness", "Facet_Immoderation", "Facet_Vulnerability",
    "Facet_Friendliness", "Facet_Gregariousness", "Facet_Assertiveness", "Facet_ActivityLevel", "Facet_ExcitementSeeking", "Facet_Cheerfulness",
    "Facet_Imagination", "Facet_ArtisticInterests", "Facet_Emotionality", "Facet_Adventurousness", "Facet_Intellect", "Facet_Liberalism",
    "Facet_Trust", "Facet_Morality", "Facet_Altruism", "Facet_Cooperation", "Facet_Modesty", "Facet_Sympathy",
    "Facet_SelfEfficacy", "Facet_Orderliness", "Facet_Dutifulness", "Facet_AchievementStriving", "Facet_SelfDiscipline", "Facet_Cautiousness"
]

# Map column name → col number for OCEAN_Facets
colnum_facets = {name: i + 1 for i, name in enumerate(header_facets)}

FLAG_COL = colnum_facets.get("FACETS_Processed", None)


# =========================
# HELPER: VALID ID CHECK
# =========================
def is_valid_id(x):
    return isinstance(x, str) and len(x) == 24


# Select rows needing processing (from GameData)
rows_to_process = [
    idx for idx, row in df_game.iterrows() if is_valid_id(row["Big5TestID"])
]


# =========================
# SCRAPER: fetch values from bigfive-test.com
# =========================
async def fetch_bigfive_data(test_id):
    url = f"https://bigfive-test.com/result/{test_id}"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)

        await page.wait_for_selector(".apexcharts-bar-area", timeout=25000)
        bars = await page.query_selector_all(".apexcharts-bar-area")

        vals = []
        for b in bars:
            raw = await b.get_attribute("val")
            if raw and raw.isdigit():
                vals.append(int(raw))

        await browser.close()

        traits = {TRAITS[i] + "_Real": vals[i] for i in range(5)}
        facets = {FACETS[i] + "_Real": vals[5 + i] for i in range(len(FACETS))}

        return traits, facets


# =========================
# MAIN LOOP
# =========================
async def process_all():
    batch = []

    for idx in rows_to_process:
        player_id = df_game.loc[idx, "PlayerID"]
        test_id = df_game.loc[idx, "Big5TestID"]

        print(f"➡ Fetching Big Five test for {test_id}")

        # Find same row in OCEAN_Facets
        row_match = df_facets[
            (df_facets["PlayerID"] == player_id) &
            (df_facets["Big5TestID"] == test_id)
        ]

        if row_match.empty:
            print(f"⚠ No matching row in OCEAN_Facets for PlayerID={player_id}")
            continue

        rownum = row_match.index[0] + 2  # +2 because header is row 1

        traits, facets = await fetch_bigfive_data(test_id)

        # --- TRAITS REAL ---
        for name, value in traits.items():
            if name in colnum_facets:
                a1 = rowcol_to_a1(rownum, colnum_facets[name])
                batch.append({"range": a1, "values": [[value]]})

        # --- FACETS REAL ---
        for name, value in facets.items():
            if name in colnum_facets:
                a1 = rowcol_to_a1(rownum, colnum_facets[name])
                batch.append({"range": a1, "values": [[value]]})

        # --- FLAG ---
        if FLAG_COL:
            a1 = rowcol_to_a1(rownum, FLAG_COL)
            batch.append({"range": a1, "values": [["TRUE"]]})

    # Send updates
    if batch:
        print(f"📦 Sending batch update: {len(batch)} updates…")
        sheet_facets.batch_update(batch)
        print("✅ OCEAN_Facets updated!")
    else:
        print("Nothing to update.")


# RUN
await process_all()

➡ Fetching Big Five test for 6903765610ac331e1bd0edc5
➡ Fetching Big Five test for 691cc5711acd3ebc317285cb
➡ Fetching Big Five test for 6907134ab2abf2e2002bc740
➡ Fetching Big Five test for 6926b7062aa721ec246267bf
➡ Fetching Big Five test for 669d25ebc89919717e7fe1bb
➡ Fetching Big Five test for 6926ee7e21c106c9c9ee3390
➡ Fetching Big Five test for 669d1e14c89919717e7fe172
📦 Sending batch update: 245 updates…
✅ OCEAN_Facets updated!


In [ ]:
# import asyncio
# import gspread
# import pandas as pd
# from playwright.async_api import async_playwright
# from google.oauth2.service_account import Credentials
# from gspread.utils import rowcol_to_a1

# # =========================
# # GOOGLE AUTH
# # =========================
# scopes = ["https://www.googleapis.com/auth/spreadsheets"]
# creds = Credentials.from_service_account_file(
#     "/Users/zentrovoy/Documents/Insight Genie/HR-GAME/My Genie game/CascadeProjects/piplines/genie-game.json",
#     scopes=scopes
# )
# client = gspread.authorize(creds)
# sheet = client.open_by_key("1lG0_Y_o2jl2rackWJyB1esAUcINb843PhNZBmTwf2L4").sheet1

# header = sheet.row_values(1)
# df = pd.DataFrame(sheet.get_all_records(expected_headers=header))

# # =========================
# # TRAIT & FACET ORDER EXACTLY LIKE SITE
# # =========================

# TRAITS = [
#     "Trait_Neuroticism",
#     "Trait_Extraversion",
#     "Trait_Openness",
#     "Trait_Agreeableness",
#     "Trait_Conscientiousness",
# ]

# FACETS = [
#     "Facet_Anxiety", "Facet_Anger", "Facet_Depression", "Facet_SelfConsciousness", "Facet_Immoderation", "Facet_Vulnerability",
#     "Facet_Friendliness", "Facet_Gregariousness", "Facet_Assertiveness", "Facet_ActivityLevel", "Facet_ExcitementSeeking", "Facet_Cheerfulness",
#     "Facet_Imagination", "Facet_ArtisticInterests", "Facet_Emotionality", "Facet_Adventurousness", "Facet_Intellect", "Facet_Liberalism",
#     "Facet_Trust", "Facet_Morality", "Facet_Altruism", "Facet_Cooperation", "Facet_Modesty", "Facet_Sympathy",
#     "Facet_SelfEfficacy", "Facet_Orderliness", "Facet_Dutifulness", "Facet_AchievementStriving", "Facet_SelfDiscipline", "Facet_Cautiousness"
# ]

# # Header → column number
# colnum = {name: i + 1 for i, name in enumerate(header)}

# FLAG_COL = colnum.get("FACETS_Processed", None)


# # =========================
# # VALID ID CHECK
# # =========================
# def is_valid_id(x):
#     return isinstance(x, str) and len(x) == 24


# rows_to_process = [
#     idx for idx, row in df.iterrows() if is_valid_id(row["Big5TestID"])
# ]

# print(f"🔍 Need to process {len(rows_to_process)} rows…")


# # =========================
# # SCRAPER (from val="")
# # =========================
# async def fetch_bigfive_data(test_id):
#     url = f"https://bigfive-test.com/result/{test_id}"

#     async with async_playwright() as p:
#         browser = await p.chromium.launch(headless=True)
#         page = await browser.new_page()
#         await page.goto(url, timeout=60000)

#         await page.wait_for_selector(".apexcharts-bar-area", timeout=25000)
#         bars = await page.query_selector_all(".apexcharts-bar-area")

#         vals = []
#         for b in bars:
#             raw = await b.get_attribute("val")
#             if raw and raw.isdigit():
#                 vals.append(int(raw))

#         await browser.close()

#         traits = {TRAITS[i]: vals[i] for i in range(5)}
#         facets = {FACETS[i]: vals[5 + i] for i in range(len(FACETS))}

#         return traits, facets


# # =========================
# # MAIN LOOP
# # =========================
# async def process_all():
#     batch = []

#     for idx in rows_to_process:
#         tid = df.loc[idx, "Big5TestID"]
#         rownum = idx + 2

#         print(f"➡ Fetching {tid}")

#         traits, facets = await fetch_bigfive_data(tid)

#         # --- TRAITS ---
#         for name, value in traits.items():
#             if name in colnum:
#                 a1 = rowcol_to_a1(rownum, colnum[name])
#                 batch.append({"range": a1, "values": [[value]]})

#         # --- FACETS ---
#         for name, value in facets.items():
#             if name in colnum:
#                 a1 = rowcol_to_a1(rownum, colnum[name])
#                 batch.append({"range": a1, "values": [[value]]})

#         # --- FLAG ---
#         if FLAG_COL:
#             a1 = rowcol_to_a1(rownum, FLAG_COL)
#             batch.append({"range": a1, "values": [["TRUE"]]})

#     if batch:
#         print(f"📦 Sending batch update: {len(batch)} updates…")
#         sheet.batch_update(batch)
#         print("✅ DONE!")
#     else:
#         print("Nothing to update")


# # RUN
# await process_all()

🔍 Need to process 7 rows…
➡ Fetching 6903765610ac331e1bd0edc5
➡ Fetching 691cc5711acd3ebc317285cb
➡ Fetching 6907134ab2abf2e2002bc740
➡ Fetching 6926b7062aa721ec246267bf
➡ Fetching 669d25ebc89919717e7fe1bb
➡ Fetching 6926ee7e21c106c9c9ee3390
➡ Fetching 669d1e14c89919717e7fe172
📦 Sending batch update: 245 updates…
✅ DONE!


In [25]:
import gspread
import pandas as pd
import numpy as np
from google.oauth2.service_account import Credentials

# 1. AUTH
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("/Users/zentrovoy/Documents/Insight Genie/HR-GAME/My Genie game/CascadeProjects/piplines/genie-game.json", scopes=scopes)
client = gspread.authorize(creds)

sheet = client.open_by_key("1lG0_Y_o2jl2rackWJyB1esAUcINb843PhNZBmTwf2L4").sheet1

header = sheet.row_values(1)
data = sheet.get_all_records(expected_headers=header)
df = pd.DataFrame(data)

if "OCEAN_Processed" not in df.columns:
    df["OCEAN_Processed"] = False

df_new = df[df["OCEAN_Processed"] != True].copy()


# ------------------------------------------------
#  v6.1 RULE-BASED RAW SCORE MODEL (upgraded v6)
# ------------------------------------------------
def psychometric_scoring_v6_1(row):

    def get_val(key, default=0.0):
        try:
            v = row.get(key, default)
            if isinstance(v, str) and v.strip() == "":
                return default
            return float(v)
        except:
            return default

    idle = get_val("IdleTime")
    dec_time = get_val("DecisionTime")
    tiles = get_val("TilesSteppedOnTrail")
    path = row.get("FirstPathChoice", "")
    coins_a = get_val("CoinsRoomA")
    high_five = get_val("HighFiveGiven")
    pumps = get_val("PumpsCount")
    exploded = get_val("BalloonExploded")
    jitter_int = row.get("JitterIntensity", "")

    # --------------------------------
    # OPENNESS (boost ×1.5)
    # --------------------------------
    op = 0
    op += min(idle, 30) * 1.2     # было 0.8
    if path == "foggy":
        op += 8                  # было 5
    if coins_a >= 4:
        op += 6                  # было 4
    if dec_time < 5:
        op += 4                  # было 2
    if 10 <= tiles <= 40:
        op += 5                  # было 3

    # --------------------------------
    # CONSCIENTIOUSNESS (boost ×2)
    # --------------------------------
    co = 0
    if exploded == 0:
        co += 20     # было 12
    elif exploded == 1:
        co += 15     # было 10
    if 20 <= dec_time <= 90:
        co += 10     # было 6
    if 15 <= tiles <= 40:
        co += 6      # было 3

    # --------------------------------
    # EXTRAVERSION (boost ×1.7)
    # --------------------------------
    ex = 0
    ex += min(idle, 60) * 0.35   # было 0.2
    if high_five == 1:
        ex += 10     # было 6
    if pumps >= 4:
        ex += 7      # было 4
    if dec_time < 15:
        ex += 5      # было 3

    # --------------------------------
    # AGREEABLENESS (boost ×2)
    # --------------------------------
    ag = 0
    ag += 14         # было 8
    if high_five == 1:
        ag += 12     # было 6
    if tiles <= 35:
        ag += 8      # было 4
    if pumps <= 5:
        ag += 6      # было 3

    # --------------------------------
    # NEUROTICISM (boost ×1.3)
    # --------------------------------
    ne = 0
    ne += min(idle, 15) * 1.3     # было 1.0
    if jitter_int == "high":
        ne += 9      # было 6
    if exploded >= 2:
        ne += 6      # было 4
    if dec_time > 100:
        ne += 6      # было 4

    return pd.Series(
        [op, co, ex, ag, ne],
        index=["Op_raw", "Co_raw", "Ex_raw", "Ag_raw", "Ne_raw"],
    )


# ------------------------------------------------
#  LOGISTIC SCALING (same as original v6)
# ------------------------------------------------
def logistic_scale(x, trait):
    cfg = {
        "Op_raw": {"center": 22, "spread": 15},
        "Co_raw": {"center": 16, "spread": 8},
        "Ex_raw": {"center": 15, "spread": 10},
        "Ag_raw": {"center": 18, "spread": 8},
        "Ne_raw": {"center": 18, "spread": 12},
    }[trait]

    z = (x - cfg["center"]) / cfg["spread"]
    prob = 1 / (1 + np.exp(-z))
    return 20 + prob * 80   # 20–100 диапазон


# ------------------------------------------------
#  APPLY TO NEW ROWS + WRITE TO SHEET
# ------------------------------------------------
if not df_new.empty:
    raw_df = df_new.apply(psychometric_scoring_v6_1, axis=1)
    df_new = pd.concat([df_new, raw_df], axis=1)

    update_data = []

    for i, row in df_new.iterrows():

        op = logistic_scale(row["Op_raw"], "Op_raw")
        co = logistic_scale(row["Co_raw"], "Co_raw")
        ex = logistic_scale(row["Ex_raw"], "Ex_raw")
        ag = logistic_scale(row["Ag_raw"], "Ag_raw")
        ne = logistic_scale(row["Ne_raw"], "Ne_raw")

        row_idx = i + 2

        update_data.append({
            "range": f"AH{row_idx}:AL{row_idx}",
            "values": [[int(op), int(co), int(ex), int(ag), int(ne)]]
        })

        update_data.append({
            "range": f"AM{row_idx}",
            "values": [["TRUE"]]
        })

    sheet.batch_update(update_data)
    print("✅ OCEAN v6.1 updated → AH–AL, AM flag set.")
else:
    print("No new rows to process.")

✅ OCEAN v6.1 updated → AH–AL, AM flag set.
